In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# NLP - review

## Loading

In [2]:
import json
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

import nltk
from nltk.tag import pos_tag
import spacy
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab

import re
from spacy.tokenizer import Tokenizer

# pip install -U spacy
# python -m spacy download en_core_web_sm
import random

# stemming
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer

# linguistic pattern
from spacy.matcher import Matcher

from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# dataset splitting
from sklearn.model_selection import train_test_split

# onehot encoding
from sklearn.preprocessing import OneHotEncoder
import collections
from argparse import Namespace
import os
import string

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from torch.nn.utils import rnn as rnn_utils

In [3]:
data_review = pd.read_csv("/content/drive/MyDrive/summer_project_2021/airbnb/reviews_full.csv")
nlp = spacy.load('en_core_web_sm')

## Data Exploration

In [4]:
data_review.head()
data_review.shape

(220969, 6)

In [5]:
data_review.keys()

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

reviews will be checked based on different year and it will be start from the latest year which is 2021.


In [6]:
data_review[['year', 'month']] = data_review['date'].str.split('-', 1, expand=True)

In [7]:
set(data_review.year)

{'2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021'}

## Process Dataset

In [8]:
df = data_review[['listing_id', 'date', 'year', 'month', 'comments']]
df.head()

,listing_id,date,year,month,comments
0,20815,2010-03-24,2010,03-24,My daughter and her friend and I all stayed at...
1,20815,2010-05-08,2010,05-08,You cannot find a better place to stay in Turk...
2,20815,2010-06-02,2010,06-02,This comfortable apartment truly deserves the ...
3,20815,2010-08-27,2010,08-27,I second all the excellent reviews of this pla...
4,20815,2010-09-03,2010,09-03,Thanks to Firat and especially Arda for their ...


In [19]:
# there are 63681 reviews in one year and due to the long execution time, I ransom select 2000 rows
review_2021 = df[df.year == '2021'].sample(n=2000).reset_index()
review_2021.head()
review_2021.shape
# 2000

(2000, 6)

In [10]:
import nltk
nltk.download('punkt') # sentence tokenize
nltk.download('average_perceptron_tagger') # POS Tagging
nltk.download('maxent_ne_chunker') # Named Entity Chunking
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Error loading average_perceptron_tagger: Package
[nltk_data]     'average_perceptron_tagger' not found in index
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### Tokenize

In [21]:
review

['El departamento era exactamente como se muestra en las fotos, tiene todo lo necesario y es muy cómodo. <br/><br/>La atención fue bastante buena y agradable. Disfrute mucho de mi estadía ahí.<br/><br/>Lo recomiendo!',
 'Harika misafirperverlik, olağanüstü konum. Ev çok güzeldi. Her şey için çok teşekkürler!',
 'Çok iyi bir ev sahibi',
 'Yardımları için ev sahibine teşekkür ederim. İletişim açısından kolaylıkla ulaşabileceğiniz birisi. Evin en üst katta olması bir dezavantaj. Ama değer :)',
 'hic sorun yasamadan konaklayabileceginiz bir yer. ev sahibi duzenli, temiz  ve iyi niyetli :)',
 'House owner is very kind, but the location is not good as you must climb the hill for 600meters to reach the tram station. Moreover, the house was a bit cold.',
 'Çok tatlı ve ilgili bir abiydi. Ev çok güzeldi 6 kişi çok rahat kaldık. Emeğinize sağlık',
 'Great location, lively area with loads of dining choices. Great transport links also.',
 'очень понравилось<br/>самый лучший вид на улицу с балкончи

In [20]:
review = list(review_2021.comments)

review_token = []
for sentence in review:
    words = nltk.word_tokenize(sentence)
    tagged_words = nltk.pos_tag(words)
    ne_tagged_words = nltk.ne_chunk(tagged_words)
    review_token = review_token + [ne_tagged_words]

TypeError: ignored